# DCOPF

In [1]:
import scipy
import sympy as smp
import numpy as np

import andes
import ams

In [2]:
andes.config_logger(stream_level=10)

In [3]:
sa = andes.load(andes.get_case('ieee14/ieee14_ace.xlsx'),
              setup=True,)

Working directory: "/Users/jinningwang/Documents/work/ams/dev/demo"
Found files: ['/Users/jinningwang/Documents/work/mambaforge/envs/ams/lib/python3.9/site-packages/andes/cases/ieee14/ieee14_ace.xlsx']
> Loaded config from file "/Users/jinningwang/.andes/andes.rc"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Input format guessed as xlsx.
Parsing input file "/Users/jinningwang/Documents/work/mambaforge/envs/ams/lib/python3.9/site-packages/andes/cases/ieee14/ieee14_ace.xlsx"...
Input file parsed in 0.3808 seconds.
Setting internal address for Bus
Setting internal address for PQ
Setting internal address for PV
Setting internal address for Slack
Setting internal address for Shunt
Setting internal address for Line
Setting internal address for Area
System internal structure set up in 0.0404 seconds.


In [4]:
ams.config_logger(stream_level=10)

In [5]:
sp = ams.load(ams.get_case('matpower/case14.m'),
              setup=True,)

Working directory: "/Users/jinningwang/Documents/work/ams/dev/demo"
Found files: ['/Users/jinningwang/Documents/work/ams/ams/cases/matpower/case14.m']
Registering parameter p0
Registering algeb p
Registering algeb q
Input format guessed as matpower.
Parsing input file "/Users/jinningwang/Documents/work/ams/ams/cases/matpower/case14.m"...
Detected raw file encoding: {'encoding': 'ascii', 'confidence': 1.0, 'language': ''}
CASE14    Power flow data for IEEE 14 bus test case.
Input file parsed in 0.0060 seconds.
System set up in 0.0014 seconds.


In [6]:
sp.routines

OrderedDict([('PFlow', Routine PFlow at 0x13eb129a0),
             ('DCPF', Routine DCPF at 0x15403b6d0),
             ('DCOPF', Routine DCOPF at 0x154042970)])

In [7]:
sp.StaticGen.n

5

In [8]:
sp.groups

OrderedDict([('ACLine', <ams.models.group.ACLine at 0x13eb1b0d0>),
             ('ACTopology', <ams.models.group.ACTopology at 0x13eb1b130>),
             ('Collection', <ams.models.group.Collection at 0x153d736d0>),
             ('Cost', <ams.models.group.Cost at 0x11efbca60>),
             ('DynLoad', <ams.models.group.DynLoad at 0x10731bb20>),
             ('Information', <ams.models.group.Information at 0x1540148e0>),
             ('StaticGen', <ams.models.group.StaticGen at 0x154014910>),
             ('StaticLoad', <ams.models.group.StaticLoad at 0x154014940>),
             ('StaticShunt', <ams.models.group.StaticShunt at 0x154014970>),
             ('Undefined', <ams.models.group.Undefined at 0x1540149a0>)])

In [9]:
sp.groups

OrderedDict([('ACLine', <ams.models.group.ACLine at 0x13eb1b0d0>),
             ('ACTopology', <ams.models.group.ACTopology at 0x13eb1b130>),
             ('Collection', <ams.models.group.Collection at 0x153d736d0>),
             ('Cost', <ams.models.group.Cost at 0x11efbca60>),
             ('DynLoad', <ams.models.group.DynLoad at 0x10731bb20>),
             ('Information', <ams.models.group.Information at 0x1540148e0>),
             ('StaticGen', <ams.models.group.StaticGen at 0x154014910>),
             ('StaticLoad', <ams.models.group.StaticLoad at 0x154014940>),
             ('StaticShunt', <ams.models.group.StaticShunt at 0x154014970>),
             ('Undefined', <ams.models.group.Undefined at 0x1540149a0>)])

In [10]:
sp.Bus.algebs

OrderedDict([('a', Algeb: Bus.a, a=[], v=[]), ('v', Algeb: Bus.v, a=[], v=[])])

In [11]:
sp.StaticGen.common_vars

['p', 'q']

In [12]:
sp.PV.p

Algeb: PV.p, a=[], v=[]

In [13]:
sp.DCOPF.p.tex_name

'p_{g}'

In [14]:
sp.DCOPF.ralgebs

OrderedDict([('p', RAlgeb: StaticGen.p, a=[], v=[]),
             ('q', RAlgeb: StaticGen.q, a=[], v=[])])

In [15]:
sp.DCOPF.rparams

OrderedDict([('p0', <ams.core.param.RParam at 0x15405a430>)])

In [16]:
sp.DCOPF.p0.group